<a href="https://colab.research.google.com/github/He-ya/freeCodeCamp/blob/master/Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART 1: 抓取PTT文本

首先先匯入必要的函式庫。
    
*   request：用來對伺服器送出特定request來抓回某頁的內容
*   BeautifulSoup：用來解析HTML檔案



In [0]:
import json
import requests
import time

from bs4 import BeautifulSoup
from bs4.element import NavigableString

首先，要抓八卦版的文章時，會跳出一個詢問是否18歲的頁面。我們要設置一個session來保持一些參數，讓我們假裝已經回答“已滿18歲“，以避這個詢問頁面跳出而打斷我們的爬蟲。這個session的資料就放在gossip_data中。

In [0]:
gossip_data = {"from":"bbs/Gossiping/index.html","yes": "yes"}

session=requests.session()
requests.packages.urllib3.disable_warnings()
session.post("https://www.ptt.cc/ask/over18",verify=False,data=gossip_data)

<Response [200]>

接下來我們要開始準備抓八卦版(gossip)的網頁。我們先用一個變數存起來版名：Gossip。並設定兩個變數start和end，分別是指定我們要從哪一頁開始爬，爬到哪一頁結束。

In [0]:
board="Gossiping"
start=1
end=20
crawl_range = range(start, end)

In [0]:
target_page = 'https://www.ptt.cc/bbs/' + board + '/index'

target_pages = []
for i in crawl_range:
    target_pages.append(target_page+str(i)+".html")

print("我抓到了這些頁：",target_pages)

我抓到了這些頁： ['https://www.ptt.cc/bbs/Gossiping/index1.html', 'https://www.ptt.cc/bbs/Gossiping/index2.html', 'https://www.ptt.cc/bbs/Gossiping/index3.html', 'https://www.ptt.cc/bbs/Gossiping/index4.html', 'https://www.ptt.cc/bbs/Gossiping/index5.html', 'https://www.ptt.cc/bbs/Gossiping/index6.html', 'https://www.ptt.cc/bbs/Gossiping/index7.html', 'https://www.ptt.cc/bbs/Gossiping/index8.html', 'https://www.ptt.cc/bbs/Gossiping/index9.html', 'https://www.ptt.cc/bbs/Gossiping/index10.html', 'https://www.ptt.cc/bbs/Gossiping/index11.html', 'https://www.ptt.cc/bbs/Gossiping/index12.html', 'https://www.ptt.cc/bbs/Gossiping/index13.html', 'https://www.ptt.cc/bbs/Gossiping/index14.html', 'https://www.ptt.cc/bbs/Gossiping/index15.html', 'https://www.ptt.cc/bbs/Gossiping/index16.html', 'https://www.ptt.cc/bbs/Gossiping/index17.html', 'https://www.ptt.cc/bbs/Gossiping/index18.html', 'https://www.ptt.cc/bbs/Gossiping/index19.html']


接著，我們先抓取Gossip版中的每一頁中的文章的網址，存在target_pages

In [0]:
article_urls = []
for page in target_pages:
    contents = []
    
    res  = session.get(page, verify=False)
    soup = BeautifulSoup(res.text, "lxml")

    #一個頁面中的每一篇文章的url
    
    for article in soup.select(".r-ent"):
        try:
            article_urls.append('https://www.ptt.cc'+ article.select(".title")[0].select("a")[0].get("href"))
        except:
            pass 

我們看一下，第一則的文章的網址和全部有幾個網址

In [0]:
print("URL:",article_urls[0])
print("總共幾則",len(article_urls))

URL: https://www.ptt.cc/bbs/Gossiping/M.1119222611.A.7A9.html
總共幾則 380


有了這些網址，我們就按照這些網址，把每一篇文章給抓下來。
抓下之後，我們可以看文章的原始檔，會發現文章的內容在#main-content這個tag下面，因此我們可以把這些字拿出來，供以後分析。這邊我們會設定文章長度的限制。因為太短的文章可能就是隨意的回文。因此如果文章長度沒有到30個字，我們就把它忽略，要大於等於30個字的文章，才會放進去我們的計算中。

In [0]:
articles=[]

for url in article_urls:
    raw  = session.get(url, verify=False)
    soup = BeautifulSoup(raw.text, "lxml")
    try:
        article=dict()
        article["title"]  = soup.select(".article-meta-value")[2].contents[0]
        content = ""
        for tag in soup.select("#main-content")[0]:
            if type(tag) is NavigableString and tag !='\n':
                content += tag
                break
                
        if len(content)>=30:
            article["content"] = content
            articles.append(article)
    except:
      pass


我們來看看每一個文章的標題吧！

In [0]:
for i in range(len(articles)):
    print(i,articles[i]['title'])

0 第一耶！
1 Re: (問題)華航空難留言
2 Re: 有沒有明天會更好的八卦
3 Re: 請問還有濁水溪公設的八卦嗎
4 Re: (問題)華航空難留言
5 [八卦] 
6 Re: [問題]職棒似乎又出現問題了...
7 Re: [政商] 請問桌伯元的八卦?
8 Re: [新聞] 張震月的八卦有嗎？
9 Re: [政商] 請問桌伯元的八卦?
10 Re: [名人] 有人記得高雄數學補教老師魚課斌嗎？
11 [藝人] 釁悅糰的阿釁
12 Re: [政商] 有羅文傢的八卦?
13 Re: 有沒有米其淋的美食還是什麼評鑑八卦
14 Re: 有沒有米其淋的美食還是什麼評鑑八卦
15 Re: [問題]職棒似乎又出現問題了...
16 Re: 誰有張憲義的八卦??
17 Re: [藝人] 小紅恩
18 Re: [政商] 有沒有前任管媒體局長林佳隆的八卦
19 Re: [名人] 翁大院長岳生先生
20 Re: [政商] 有沒有前任管媒體局長林佳隆的八卦
21 [八卦] 海洋音樂祭的八卦
22 Re: 有沒有駕訓班的八卦？
23 Re: [八卦] 海洋音樂祭的八卦
24 Re: [八卦] 謝長停的八卦
25 Re: [八卦] 謝長停的八卦
26 [藝人] 理欠蓉的小八卦
27 Re: 南加大的全名是什麼?
28 Re: [政商] 有沒有前任管媒體局長林佳隆的八卦
29 Re: [新聞] 張震月的八卦有嗎？
30 Re: 請問有維也納少年合唱團的八卦嗎
31 Re: [政商] 有沒有前任管媒體局長林佳隆的八卦
32 [八卦] 去國外的八卦
33 Re: [八卦] 說一下沈副熊的八卦
34 Re: [八卦] 說一下沈副熊的八卦
35 Re: [八卦] 去國外的八卦
36 Re: [藝人] 賈靜雯到底有沒有被騙？
37 Re: 關於最近台日兩方漁業糾紛導致的反日內幕
38 Re: [名人] 好萊塢的女星也有陪睡的情況嗎
39 Re: [八卦] 去國外的八卦
40 Re: 一些男藝人的性向八卦
41 Re: [八卦] 說一下沈副熊的八卦
42 Re: [名人] 有毆楊隆或副涓的八卦嗎?
43 Re: [政商] 那我可不可以問一下陳奇邁的八卦? 
44 Re: 請問港星Grace葉佩雯
45 Re: [前藝人] 有BOBO的八卦嗎  身材很好那個
46 Re: 記得那個賣牛肉面辦大學的?

太好了！我們成功將PTT的八卦版文章抓下來了

# PART 2: 找出相似文章

首先我們要安裝jieba函式庫，並下載繁中字典

In [0]:
!pip install jieba

In [0]:
!wget https://www.dropbox.com/s/ikv3n0fzb218vgn/dict.txt.big

--2018-12-26 16:24:18--  https://www.dropbox.com/s/ikv3n0fzb218vgn/dict.txt.big
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ikv3n0fzb218vgn/dict.txt.big [following]
--2018-12-26 16:24:18--  https://www.dropbox.com/s/raw/ikv3n0fzb218vgn/dict.txt.big
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc98b397544944bd83417e3ab22e.dl.dropboxusercontent.com/cd/0/inline/AYPLOm7WObt4-kb4voPGvKGNHnAjsdN4uULnxiRwoBI0XO8jUPrD-VIVzg2YjtyEc53DdHVm2M-u92xsERPHTJLYjy9Wqte76mAnVWx2clkAfTdNnTsge-16PUsPBx3nxKAgMDlmHAVroc-ekv8OEXMsuaCSvkXsFhuZR-zfJpr_8YKeYqtsVWWyc1SOoikFWV4/file [following]
--2018-12-26 16:24:19--  https://uc98b397544944bd83417e3ab22e.dl.dropboxusercontent.com/cd/0/inline/AYPLOm7WObt4-kb4voPGvKGNHnAjsdN4uULnxiRwoBI0XO8jUPr

首先我們引入jieba函式庫，這是個用來處理中文斷詞的函式庫。

In [0]:
import jieba
import jieba.analyse

接著，我們載入繁中的字典

In [0]:
jieba.set_dictionary('dict.txt.big')

現在我們要把每一篇文章的關鍵字求出來。keywords是一個list，其中第1個元素就是第1篇文章的關鍵字們，依此類推。

In [0]:
keywords = []

for i in range(len(articles)):
    keywords.append(jieba.analyse.extract_tags(articles[i]['content'],topK=20, withWeight=True))

來看看第一篇文章的關鍵字！

In [0]:
print(keywords[0])

[('Beauty', 0.27801784890465114), ('娛樂', 0.27801784890465114), ('二度', 0.2280542579011628), ('Gossiping', 0.13900892445232557), ('綜合', 0.13900892445232557), ('記者', 0.13900892445232557), ('抄襲', 0.13900892445232557), ('HOTlinkinpork', 0.13900892445232557), ('Hate', 0.13900892445232557), ('期末考', 0.13900892445232557), ('HOTloverlover', 0.13900892445232557), ('ci', 0.13900892445232557), ('FORMULA1', 0.13900892445232557), ('賽車', 0.13900892445232557), ('Sad', 0.13900892445232557), ('GRAND', 0.13900892445232557), ('PRIX', 0.13900892445232557), ('HOTMSCHUMACHER', 0.13900892445232557), ('sex', 0.13900892445232557), ('原來', 0.13900892445232557)]


接著，我們將對每一篇文章有意義的字串起來，把他視為是該文章的總結

In [0]:
fenci_str=[]

for x in keywords:
    keys = ""
    for i in range(len(x)):
        keys = keys + str(x[i][0])
        if i!=len(x)-1:
            keys = keys + " "
    fenci_str.append(keys)

In [0]:
fenci_str

['Beauty 娛樂 二度 Gossiping 綜合 記者 抄襲 HOTlinkinpork Hate 期末考 HOTloverlover ci FORMULA1 賽車 Sad GRAND PRIX HOTMSCHUMACHER sex 原來',
 '讀者文摘 737 沒有 後來 這架 客機 油料 滑翔翼 這推文 一個 飛機 沒油 現憑 寫文 有錯 報告 中是 導致 加滿 架飛機',
 '反盜版 當年 捐給 消基會 理由 捐錢 給消 基會 美國 WORLD 之後 台灣 這些 覺得 如法泡製 國家 人民富足 實在 於是 拿來',
 '鼓手 濁水 小柯 濁水溪 紀錄片 左派 後來 ... 貝斯手 離團 場合 音樂 問題 高尚 公社 吉他手 早期 霹靂 四號 時候',
 '上將 總司令 直升機 空難 陸軍 當時 官階 還有 不過 將軍 這位 上尉 講到 民國 63 演習 曾經 發生 有史以來 嚴重',
 '張大帥 台灣 球隊 兩次 他們 友人 職棒 總共 隊伍 主場 火鍋店 光顧 幾次 親眼看到 身邊 一樣 後援會 幫忙 豐富 萬人迷',
 '新竹 顧客 球衣 綠色 我們 母公司 橘色 執業 理髮師 一個 職棒 組頭 給我 根據 每隊 每個 地區 一樣 上來 農藥',
 '井田 縣長 立委 最後 彰化縣 國民黨 桌伯元 KMT 因為 文宣 還有 沒有 伯元 的詩 有點 來說 縣議員 當選 葉津諷 不過',
 '記者 有沒有 覺得 出來 路加宜 媽媽 這樣 巴拉巴 聽到 問張 結婚 很煩 變成 幫張 據說 報導 諒解 生氣 質問 張為',
 '大學 國立 彰化縣 八卦 桌伯元 法學碩士 台灣 學士 海洋大學 法學 碩士 縣長 副縣長 國民黨 團副 書記長 議員 兩屆 靜宜 及大葉',
 '據說 同學 醫科 超醫班 後來 聯考 第一班 數學 名師 發現 書卷 畢業 重考 一屆 前立行 最後 榜單 上課 當年 馬上',
 '阿釁 然後 筱划 時候 現在 要講 po 有關 團費 賭博 pub 或許 因為 對著 我們 寰還 結果 這時 那時快 頭上',
 '發現 算不算 八卦 部落 知道 就是',
 '米其林 輪胎 手冊 發行 自動車 自行車 機車 車主 何處 更換 1926 他們 創造 獨有 星級 不過 這時 變成 只賣 帶來',
 '米其林 餐廳 Gault Millau 評

積著，我們要利用scikit learn中的TfidfTransformer將文章的TF-IDF特徵向量算出來。

先引入函式庫們：

In [0]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

接著，先用CountVectorizer將每篇文章的總結處理過，再用TfidfTransformer將每一篇文章轉成特徵向量。

In [0]:
transformer = TfidfTransformer()
cv = CountVectorizer()
term_doc = cv.fit_transform(fenci_str)
fv = transformer.fit_transform(term_doc).toarray()

來看一下特徵向量長怎樣

In [0]:
fv[0]

array([0., 0., 0., ..., 0., 0., 0.])

算出每一個文章的特徵向量之後，重頭戲來了。我們要算出與每一篇文章的相似文章有誰。這邊我們要用的函式庫是NearestNeighbors。顧名思義，就是要找最近的鄰居。因為這邊考慮的是文章的特徵向量，因此就是找最相近的文章的意思。

In [0]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

我們利用NearestNeighbors函數，找出與某篇文章最像的k篇文章。我們這邊是只要看最像的2篇文章就好(n_neighbors=2)。其中，輸出的兩個項中，distances是與該篇文章的距離，indices是最像的文章的編號

In [0]:
nbrs = NearestNeighbors(n_neighbors=2,algorithm='ball_tree').fit(fv)
distances, indices = nbrs.kneighbors(fv)

看看結果！

In [0]:
indices

array([[  0, 226],
       [  1, 332],
       [  2, 159],
       [  3,  66],
       [  4, 132],
       [  5,   6],
       [  6,   5],
       [  7,   9],
       [  8,  67],
       [  9,   7],
       [ 10, 260],
       [ 11, 213],
       [ 12, 271],
       [ 13,  11],
       [ 14, 149],
       [ 15,  39],
       [ 16,  87],
       [ 17, 131],
       [ 18, 260],
       [ 19, 308],
       [ 20,  44],
       [ 21, 131],
       [ 22,  67],
       [ 23, 170],
       [ 24, 173],
       [ 25,  60],
       [ 26, 260],
       [ 27,  75],
       [ 28,  44],
       [ 29,  18],
       [ 30, 261],
       [ 31, 314],
       [ 32,  39],
       [ 33, 179],
       [ 34,  39],
       [ 35,  39],
       [ 36, 246],
       [ 37, 167],
       [ 38, 138],
       [ 39,  32],
       [ 40,  29],
       [ 41, 223],
       [ 42, 106],
       [ 43,  17],
       [ 44, 200],
       [ 45, 231],
       [ 46,  39],
       [ 47, 137],
       [ 48, 137],
       [ 49, 151],
       [ 50, 233],
       [ 51,  41],
       [ 52,

這個結果的解讀方法是：與第i篇文章像的，就是第i個元素。所以可以看到，indices[1] = [1.332]。意思就是說，與編號1文章像的，是編號1和332的文章。（因為自己一定和自己最像，所以編號1一定在裡面。）我們來看看兩篇文章內容！

In [0]:
print(articles[1]['content'])


 看到這推文,忍不住要寫一個飛機沒油的例子
 很久之前在讀者文摘上看到,現憑印象寫文,有錯請指正

 某家航空飛加拿大的737,飛到半途沒有燃料,
 後來報告中是說添加油料時,把公制和英制混淆,
 導致那架737的油料沒加滿

 這架飛機沒有墜毀,
 因為駕駛員使用自己飛滑翔翼的經驗和技巧,
 平安地讓這架乘坐上百人的大型客機,以滑翔的方式降落到附近的機場

 後來這架客機沒多大損傷,還可以繼續服務,被業界暱稱為"滑翔機"

 駕駛真神人 =.=
---
 每次翻讀者文摘都會找這篇故事出來看 ^^
--



In [0]:
print(articles[332]['content'])


我在分享幾個在節目上看到的笑話

都是九恐在空軍官校的事

故事一

九恐有一天出飛行訓練

因為是有編組的

然後飛在九恐旁邊的飛機裡面的教官

就把雙手放開給九恐後面的教官看

表示都是他的學生自己開的

有點挑釁的意味

結果九恐要幫教官出頭

說: 教官你放手 讓我來 (正經)

結果....


教官就用一隻手手刀狂打的他頭


說

你有本事跟人家講這種話嗎?(怒)............XD

故事二

九恐的飛行技術很爛

但是很會背一些飛行教條的東西

比如說方位00 放起落架.....

然後他說要降落的時候要"對三邊"(不太清楚)

(他上節目背的超流利)

然後開始準備降落

就是要對正跑道的意思

然後

他怎麼對都對不到跑道

他一直想不通

口訣都背的很好

但是就是對不到

最後

教官就說

你看到機場旁邊的煙囪了沒

對正他就好

後來九恐終於很順利的對正

這天

他又出飛行任務

然後.........

要回航的時候

他又沒辦法對正了

教官就罵他說

我不是教過你了麻

九恐就說

報告教官

我找不到煙囪

因為今天是假日他停工....................XD


後記

九恐的方向感很差

比如說他是嘉義空軍

卻常常飛到屏東基地去

讓當地的塔台哭笑不得

有一次出任務

他又迷路了

飛到高速公路上空

他的教官忍不住了

九恐阿

你看到路上的車子了沒

九恐說 報告教官 看到了

教官說 看仔細了喔

.......................


教官求你 不要再開飛機了 去開卡車好不好 OTZ...........

完

個人真的蠻欣賞他的

字又寫的漂亮........

--



都是在講飛行的！滿好的！

如果你要建立一個推薦系統，你也可以把NearestNeighbors()中的參數n_neighbors調大，這樣你就可以找出跟某篇文章很相像的數篇文章囉！！
